# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [ ]:
# Import this library
#from nbacktest import Backtest, Strategy
from nbacktest.nbacktest import Backtest, Strategy
# Import other libraries
import yfinance as yf

# Ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from nbacktest.nbacktest import Backtest, Strategy

Exception: File `'nbacktest/nbacktest.py'` not found.

## <b> Create Strategy </b>

In [2]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.broker.balance/2)/self.price["AAPL"])
        quantity2 = int((self.broker.balance/2)/self.price["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download df_ohlc Data from Yahoo Finance </b>

In [3]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  130.173782  89.279305  130.726619  89.567592  127.982170   
2022-12-27  128.367172  87.410416  129.729530  88.971138  127.073927   
2022-12-28  124.428200  85.949104  129.354385  87.996929  124.260376   
2022-12-29  127.952576  88.424385  128.811445  88.836934  126.096619   
2022-12-30  128.268448  88.205696  128.288196  88.305104  125.800425   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  87.102251  129.245801  87.102251  63814900  17815000  
2022-12-27  87.017753  129.699914  88.782258  69007800  15470900  
2022-12-28  85.859640  128.011777  86.982960  85438400  17879600  
2022-12-29  86.475968  126.353289  86.515732  75703700  18280700  
2022-12-30  86.515737  126.767896  86.848756  77034200  19190300

## <b> Run Backtest </b>

In [4]:
bt = Backtest(data=df_ohlc, universe=universe, strategy=TestStrategy, price_column="Close", cash=100_000)

result = bt.run()

Strategy started!
"['ITERATION'] not found in axis"


KeyError: "['ITERATION'] not found in axis"

In [ ]:
result.head()

Price           Close                  High                   Low             \
Ticker           AAPL       GOOG       AAPL       GOOG       AAPL       GOOG   
Date                                                                           
2018-01-02  40.426823  52.935345  40.436212  53.031772  39.722768  51.952687   
2018-01-03  40.419785  53.804176  40.964255  53.993549  40.356422  52.846369   
2018-01-04  40.607536  53.999020  40.710798  54.355403  40.384586  53.879828   
2018-01-05  41.069870  54.785843  41.156702  54.886248  40.612235  54.277364   
2018-01-08  40.917324  55.019951  41.213026  55.235171  40.818753  54.755524   

Price            Open                Volume           _iteration  \
Ticker           AAPL       GOOG       AAPL      GOOG              
Date                                                               
2018-01-02  39.933986  52.107267  102223600  24752000          0   
2018-01-03  40.490191  52.901044  118071600  28604000          1   
2018-01-04  40.492539  54.078549   89738400  20092000          2   
2018-01-05  40.703762  54.376775   94640000  25582000          3   
2018-01-08  40.917324  54.785842   82271200  20952000          4   

Price             _equity _buy _sell _trade  
Ticker                                       
Date                                         
2018-01-02  100000.000000  1.0   NaN    NaN  
2018-01-03  100811.477951  NaN   NaN    NaN  
2018-01-04  101227.471207  NaN   NaN    NaN  
2018-01-05  102541.676804  NaN   NaN    NaN  
2018-01-08  102574.127960  NaN   NaN    NaN

## <b> Backtest Statistics </b>

In [ ]:
print("---------------------------")
print("Final balance: %.2f" % bt.broker.balance)
print("Final equity: %.2f" % bt.broker.equity)
print("---------------------------")

bt.broker.df_orderbook

---------------------------
Final balance: 61.48
Final equity: 241867.46
---------------------------


,ITERATION,ACTION,TICKER,QUANTITY,PRICE,COMMISSION,SLIPPAGE,TOTAL,STATUS
ID,,,,,,,,,
0,0,buy,AAPL,1236,40.426823,0,0,-49967.552811,filled
1,0,buy,GOOG,944,52.935345,0,0,-49970.965393,filled


In [ ]:
bt.broker.df_positions

,QUANTITY,COMMISSION,SLIPPAGE,TOTAL_INVESTED,VALUE
TICKER,,,,,
AAPL,1236,0,0,-49967.552811,158539.801575
GOOG,944,0,0,-49970.965393,83266.177124


In [ ]:
bt.broker.df_tradebook

,STATUS,DESCRIPTION,PNL,CREATED_AT_ITERATION,CLOSED_AT_ITERATION,STOP_LOSS,TAKE_PROFIT,MAX_AGE
ID,,,,,,,,
